In [17]:
#importer les packages
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

from utils.load_corrections import load_all_correctifs
from utils.opendata import opendata_atlas

In [ ]:
CORRECTIFS_dict_esr = load_all_correctifs('_esr')

à faire tourner pour chaque année:

In [2]:
rentree_sco='2023'

In [3]:
df=pd.read_json(f"./POST_GENTAB/atlas{rentree_sco}.json")

In [ ]:
#on verfifie que tous les codes d'uu sont identifiés
df[pd.isna(df.UUCR_ID)]

In [ ]:
#on vérifie que les COMPOS ne valent pas les codes communes (drom et com pas grave, 97)
df[df.apply(lambda x:x.COMPOS[:5]==x.COM_CODE ,axis=1)][['COMPOS','COM_CODE']].drop_duplicates()

In [ ]:
#on verfifie que tous les secteurs sont remplis
df[pd.isna(df.SECT)]

In [ ]:
#on verfifie que tous les sexes sont remplis
df[pd.isna(df.SEXE)]

In [ ]:
df=opendata_atlas(df,rentree_sco)

In [19]:
dict_rentree={}
dict_annee={}
for year in range(2001,int(datetime.now().year)):
    dict_annee[str(year)]=str(year + 1)
    if year < 2009:
        dict_rentree[str(year)]=f"{year}-0{int(str(year)[2:])+1}"
    else:
        dict_rentree[str(year)]=f"{year}-{int(str(year)[2:])+1}"
rentree=pd.DataFrame(dict_rentree.items(), columns=['RENTREE', 'annee_universitaire'])
annee=pd.DataFrame(dict_annee.items(), columns=['RENTREE', 'annee'])

In [21]:
NIVEAU=pd.DataFrame({"ACADEMIE":"Académie","COMMUNE":"Commune","DEPARTEMENT":"Département","PAYS":"Pays","REGION":"Région","UNITE_URBAINE":"Unité urbaine"}.items(), columns=['NIVEAU', 'niveau_geographique'])
rgp=pd.DataFrame({"CPGE":"Classes préparatoires aux grandes écoles (CPGE)","EC_ART":"Écoles supérieures art et culture","EC_autres":"Autres écoles de spécialités diverses","EC_COM":"Écoles de commerce, gestion et comptabilité","EC_JUR":"Écoles juridiques et administratives","EC_PARAM":"Écoles paramédicales et sociales","ENS":"Écoles normales supérieures (ENS)","GE":"Grands établissements MENESR","ING_autres":"Autres formations d'ingénieurs","INP":"Instituts nationaux polytechniques (INP)","IUFM":"Instituts universitaires de formation des maîtres","STS":"Sections de techniciens supérieurs (STS) et assimilés","TOTAL":"Total des formations d'enseignement supérieur","UNIV":"Universités","EPEU":"Établissements d'enseignement universitaire privés","UT":"Universités de technologie (UT)"}.items(), columns=['RGP3', 'rgp_formations_ou_etablissements'])
sect=pd.DataFrame({"PU":"Établissements publics","PR":"Établissements privés"}.items(), columns=['SECT', 'secteur_de_l_etablissement'])
sexe=pd.DataFrame({"1":"Masculin","2":"Feminin"}.items(), columns=['SEXE', 'sexe_de_l_etudiant'])
niveau=['PAYS','REGION','ACADEMIE','DEPARTEMENT','UNITE_URBAINE','COMMUNE']
tab=['PAYS','REGIONS','ACADEMIES','DEPARTEMENTS','UNITE_URBAINES','COMMUNES']
varid=[['PAYS_ID'],['REG_ID'],['ACA_ID'],['DEP_ID'],['UUCR_ID'],['COM_CODE']]
varlib=['PAYS_NOM','REG_NOM','ACA_NOM','DEP_NOM','UUCR_NOM','COM_NOM']
geolist=['','REG','REG ACA','REG ACA DEP','REG UUCR','REG ACA DEP UUCR']
geolist2=['','reg_id','reg_id aca_id','reg_id aca_id dep_id','reg_id uucr_id','reg_id aca_id dep_id uucr_id']
geolist3=['','reg_nom','reg_nom aca_nom','reg_nom aca_nom dep_nom','reg_nom uucr_nom','reg_nom aca_nom dep_nom uucr_nom']
df['REG']=df.loc[:,'REG_ID']
df['ACA']=df.loc[:,'ACA_ID']
df['DEP']=df.loc[:,'DEP_ID']
df['UUCR']=df.loc[:,'UUCR_ID']
dict_a={'RENTREE':'rentree','EFFTOT':'effectif','EFFSDC':'effectifhdccpge','SEXE':'sexe','SECT':'secteur','RGP3':'regroupement','PAYS_ID':'geo_id','REG_ID':'geo_id','ACA_ID':'geo_id','DEP_ID':'geo_id','UUCR_ID':'geo_id','COM_CODE':'geo_id'}
dict_b={'PAYS_NOM':'geo_nom','REG_NOM':'geo_nom','ACA_NOM':'geo_nom','DEP_NOM':'geo_nom','UUCR_NOM':'geo_nom','COM_NOM':'geo_nom'}
dict_c={'EFFTOT':'effectif_dut'}
dict_d={'EFFTOT':'effectif_ing'}
dict_e={'EFFTOT':'effectif_form_ens'}
dict_g={'EFF_STS_APP':'effectif_apprentis_en_sts'}
dict_f={'ACA':'aca_id','DEP':'dep_id','UUCR':'uucr_id','REG':'reg_id'}
df['RENTREE']=df['RENTREE'].astype('str')
df['SEXE']=df['SEXE'].astype('str')
communes=pd.DataFrame(CORRECTIFS_dict_esr['LES_COMMUNES']).drop_duplicates()
communes['pays']=communes.loc[:,'REG_ID'].apply(lambda x: 'Étranger' if x=='R99' else 'France')

In [ ]:
for i in range(6):
    if 'EFFSDC' not in df.columns:
        df['EFFSDC']=0
    if 'EFF_STS_APP' not in df.columns:
        df['EFF_STS_APP']=0
    print(i)
    if i==0:
        t1=df.groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t2=df.loc[df['ING']=='ING'].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t3=df.loc[df['IUT']=='IUT'].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t4=df.loc[df['INSPE']=='ESPE'].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t5=df.loc[df['EFF_STS_APP']!=0].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFF_STS_APP':'sum'}) 
        
        t11=t1.groupby(['RENTREE','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t22=t2.groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t33=t3.groupby(['RENTREE','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t44=t4.groupby(['RENTREE','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t55=t5.groupby(['RENTREE','SECT','SEXE']+varid[i], as_index=False, dropna=False).agg({'EFF_STS_APP':'sum'}) 
    else:
        t1=df.groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i]+list(geolist[i].split(' ')), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t2=df.loc[df['ING']=='ING'].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t3=df.loc[df['IUT']=='IUT'].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t4=df.loc[df['INSPE']=='ESPE'].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t5=df.loc[df['EFF_STS_APP']!=0].groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFF_STS_APP':'sum'}) 


        t11=t1.groupby(['RENTREE','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t22=t2.groupby(['RENTREE','RGP3','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t33=t3.groupby(['RENTREE','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t44=t4.groupby(['RENTREE','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFFTOT': 'sum','EFFSDC': 'sum'}) 
        t55=t5.groupby(['RENTREE','SECT','SEXE']+varid[i]+geolist[i].split(' '), as_index=False, dropna=False).agg({'EFF_STS_APP':'sum'}) 
    
    t111=pd.concat([t1,t11])
    t111.loc[pd.isna(t111['RGP3']),'RGP3']='TOTAL'
    t111['NIVEAU']=[niveau[i] for x in range(0,len(t111))]
    t111=pd.merge(t111,rentree,on='RENTREE',how='left')
    t111=pd.merge(t111,annee,on='RENTREE',how='left')
    t111=pd.merge(t111,NIVEAU,on='NIVEAU',how='left')
    t111=pd.merge(t111,rgp,on='RGP3',how='left')
    t111=pd.merge(t111,sect,on='SECT',how='left')
    t111=pd.merge(t111,sexe,on='SEXE',how='left')
    t111['niveau_geo']=t111.loc[:,'NIVEAU']

    t222=pd.concat([t2,t22])
    t222.loc[pd.isna(t222['RGP3']),'RGP3']='TOTAL'

    t333=pd.concat([t3,t33])
    t333.loc[pd.isna(t333['RGP3']),'RGP3']='TOTAL'

    t444=pd.concat([t4,t44])
    t444.loc[pd.isna(t444['RGP3']),'RGP3']='TOTAL'

    t555=pd.concat([t5,t55])
    t555.loc[pd.isna(t555['RGP3']),'RGP3']='TOTAL'

    if i==0:
        A=['RENTREE','annee_universitaire','annee','niveau_geo','niveau_geographique','RGP3','rgp_formations_ou_etablissements','SECT','secteur_de_l_etablissement','SEXE','sexe_de_l_etudiant','EFFTOT','EFFSDC']+varid[i]
        df1=t111[A]
        df1['pays']=df1.loc[:,'PAYS_ID'].apply(lambda x: 'Étranger' if x=='PAYS_999' else 'France')
        df1=df1.rename(columns=dict_a)
        df1=df1.rename(columns=dict_b)
        print(df1.columns)
    else:
        A=['RENTREE','annee_universitaire','annee','niveau_geo','niveau_geographique','RGP3','rgp_formations_ou_etablissements','SECT','secteur_de_l_etablissement','SEXE','sexe_de_l_etudiant','EFFTOT','EFFSDC']+varid[i]+geolist[i].split(' ')
        T=t111[A].rename(columns=dict_a)
        C=communes[[varlib[i],varid[i][0],'pays']].rename(columns=dict_b).drop_duplicates()
        df1=pd.merge(T,C,how= 'left', left_on='geo_id', right_on=varid[i][0])
        del df1[varid[i][0]]
        print(df1.columns)
    
    df2=pd.merge(df1,t333[['EFFTOT','RGP3','SECT','SEXE',varid[i][0]]].rename(columns=dict_c).drop_duplicates(),how= 'left', left_on=['geo_id','regroupement','secteur','sexe'], right_on = [varid[i][0],'RGP3','SECT','SEXE'])
    df2=df2.drop([varid[i][0],'RGP3','SECT','SEXE'], axis=1)
    df3=pd.merge(df2,t222[['EFFTOT','RGP3','SECT','SEXE',varid[i][0]]].rename(columns=dict_d).drop_duplicates(),how= 'left', left_on=['geo_id','regroupement','secteur','sexe'], right_on = [varid[i][0],'RGP3','SECT','SEXE'])
    df3=df3.drop([varid[i][0],'RGP3','SECT','SEXE'], axis=1)
    df4=pd.merge(df3,t444[['EFFTOT','RGP3','SECT','SEXE',varid[i][0]]].rename(columns=dict_e).drop_duplicates(),how= 'left', left_on=['geo_id','regroupement','secteur','sexe'], right_on = [varid[i][0],'RGP3','SECT','SEXE'])
    df4=df4.drop([varid[i][0],'RGP3','SECT','SEXE'], axis=1)
    df5=pd.merge(df4,t555[['EFF_STS_APP','RGP3','SECT','SEXE',varid[i][0]]].rename(columns=dict_g).drop_duplicates(),how= 'left', left_on=['geo_id','regroupement','secteur','sexe'], right_on = [varid[i][0],'RGP3','SECT','SEXE'])
    df5=df5.drop([varid[i][0],'RGP3','SECT','SEXE'], axis=1)

    df5['geo_id']=df5.loc[:,'geo_id'].apply(lambda x: 'PAYS_AUTRE' if x=='PAYS_999' else x)

    df5['effectif_dut']=df5.loc[:,'effectif_dut'].apply(lambda x: 0 if pd.isna(x) else x)
    df5['effectif_ing']=df5.loc[:,'effectif_ing'].apply(lambda x: 0 if pd.isna(x) else x)
    df5['effectif_form_ens']=df5.loc[:,'effectif_form_ens'].apply(lambda x: 0 if pd.isna(x) else x)
    df5['effectif_apprentis_en_sts']=df5.loc[:,'effectif_apprentis_en_sts'].apply(lambda x: 0 if pd.isna(x) else x)


    if i==0:
        df6=df5.drop_duplicates(subset=['rentree', 'annee_universitaire', 'annee', 'niveau_geo','niveau_geographique', 'regroupement',
        'rgp_formations_ou_etablissements', 'secteur',
        'secteur_de_l_etablissement', 'sexe', 'sexe_de_l_etudiant', 'effectif',
        'effectifhdccpge','geo_id','pays'])

        df7=df6.groupby(['rentree', 'annee_universitaire', 'annee', 'niveau_geo','niveau_geographique', 'regroupement',
            'rgp_formations_ou_etablissements', 'secteur',
            'secteur_de_l_etablissement', 'sexe', 'sexe_de_l_etudiant', 'geo_id','pays'], as_index=False, dropna=False).agg({'effectif': 'sum','effectifhdccpge': 'sum','effectif_dut': 'sum','effectif_ing': 'sum','effectif_form_ens': 'sum','effectif_apprentis_en_sts':'sum'}) 
    
    else:
        l1=['rentree', 'annee_universitaire', 'annee', 'niveau_geo','niveau_geographique', 'regroupement',
        'rgp_formations_ou_etablissements', 'secteur',
        'secteur_de_l_etablissement', 'sexe', 'sexe_de_l_etudiant', 'effectif',
        'effectifhdccpge','effectif_apprentis_en_sts','geo_id', 'geo_nom']+geolist2[i].split(' ')+['pays']
        df6=df5.rename(columns=dict_f).drop_duplicates(subset=l1)

        l2=['rentree', 'annee_universitaire', 'annee', 'niveau_geo','niveau_geographique', 'regroupement',
            'rgp_formations_ou_etablissements', 'secteur',
            'secteur_de_l_etablissement', 'sexe', 'sexe_de_l_etudiant', 'geo_id', 'geo_nom']+geolist2[i].split(' ')+['pays']
        df7=df6.groupby(l2, as_index=False, dropna=False).agg({'effectif': 'sum','effectifhdccpge': 'sum','effectif_dut': 'sum','effectif_ing': 'sum','effectif_form_ens': 'sum','effectif_apprentis_en_sts':'sum'}) 

    df7['a_des_effectifs_dut']=df7.loc[:,'effectif_dut'].apply(lambda x: 'oui' if x>0 else 'non')
    df7['a_des_effectifs_ing']=df7.loc[:,'effectif_ing'].apply(lambda x: 'oui' if x>0 else 'non')
    df7['a_des_effectifs_form_ens']=df7.loc[:,'effectif_form_ens'].apply(lambda x: 'oui' if x>0 else 'non')
    df7['a_des_effectifs_apprentis_sts']=df7.loc[:,'effectif_apprentis_en_sts'].apply(lambda x: 'oui' if x>0 else 'non')

    df8=df7[df7.effectif!=0]
    if i==4:
        df8['uu']=df8['geo_id'].apply(lambda x: True if str(x)[:2]=='UU' else False)
        df8=df8[df8['uu']==True]
        del df8['uu']
    niveau[i]=df8
    


In [190]:
#concat tous les niveaux de geo_id

In [221]:
df_all=pd.concat([niveau[0],niveau[1],niveau[2],niveau[3],niveau[4],niveau[5]])
df_all.index=[x for x in range(len(df_all))]

In [193]:
df_all=df_all[['rentree', 'annee_universitaire', 'annee', 'regroupement','niveau_geo','niveau_geographique',
       'rgp_formations_ou_etablissements', 'secteur',
       'secteur_de_l_etablissement', 'sexe', 'sexe_de_l_etudiant', 'geo_id',
       'effectif', 'effectifhdccpge', 'effectif_dut', 'effectif_ing',
       'effectif_form_ens', 'effectif_apprentis_en_sts', 'a_des_effectifs_dut',
       'a_des_effectifs_ing', 'a_des_effectifs_form_ens',
       'a_des_effectifs_apprentis_sts', 'geo_nom', 'pays', 'reg_id', 'aca_id',
       'dep_id', 'uucr_id']]

In [194]:
df_all.to_json(f"./OUTPUT_OPENDATA/atlas{rentree_sco}.json", orient='records')

Une fois qu'on a fait tourner ce qu'il y a au dessus pour chaque année (à moins qu'il n'y ait pas de changement dans le code, on peut garder les output des années précédentes j'imagine), on concatene toutes les années ensemble

In [12]:
df1=pd.read_json(f"./OUTPUT_OPENDATA/atlas2001.json")
for i in range(2002,int(rentree_sco)+1,1):
    df2=pd.read_json(f"./OUTPUT_OPENDATA/atlas{i}.json")
    df1=pd.concat([df1,df2])

In [13]:
df=df1.sort_values(by=["rentree"], ascending=False)

In [14]:
df_tot=df.reset_index()
del df_tot['index']

In [15]:
df_com=df_tot

In [16]:
del df_tot['pays']

In [17]:
df_tot.to_json("./OUTPUT_OPENDATA/OD_atlas_all.json", orient='records')

In [18]:
df_tot.to_csv("./OUTPUT_OPENDATA/OD_atlas_all.csv", sep=";", encoding="UTF-8", index=False)

juste les communes

In [19]:
df_com=df_com[(df_com.niveau_geo=='COMMUNE')&(df_com.regroupement!='TOTAL')].rename(columns={"geo_id":"com_id", "geo_nom":"com_nom"})

In [22]:
df_com=pd.merge(df_com,communes[['COM_CODE','UUCR_NOM','DEP_NOM','ACA_NOM','REG_NOM']], how='left', left_on='com_id', right_on='COM_CODE')

In [23]:
del df_com['niveau_geo']
del df_com['niveau_geographique']

In [24]:
'pays' in list(df_com.columns) 

False

In [25]:
df_com.to_json(f"./OUTPUT_OPENDATA/OD_atlas_all_com.json", orient='records')
df_com.to_csv("./OUTPUT_OPENDATA/OD_atlas_all_com.csv", sep=";", encoding="UTF-8", index=False)

In [26]:
df_com

,rentree,annee_universitaire,annee,regroupement,rgp_formations_ou_etablissements,secteur,secteur_de_l_etablissement,sexe,sexe_de_l_etudiant,com_id,...,com_nom,reg_id,aca_id,dep_id,uucr_id,COM_CODE,UUCR_NOM,DEP_NOM,ACA_NOM,REG_NOM
0,2023,2023-24,2024,UT,Universités de technologie (UT),PU,Établissements publics,2,Feminin,90094,...,Sevenans,R27,A03,D090,CR90094,90094,Sevenans,Territoire de Belfort,Besançon,Bourgogne-Franche-Comté
1,2023,2023-24,2024,CPGE,Classes préparatoires aux grandes écoles (CPGE),PR,Établissements privés,1,Masculin,63113,...,Clermont-Ferrand,R84,A06,D063,UU63701,63113,Clermont-Ferrand,Puy-de-Dôme,Clermont-Ferrand,Auvergne-Rhône-Alpes
2,2023,2023-24,2024,CPGE,Classes préparatoires aux grandes écoles (CPGE),PR,Établissements privés,1,Masculin,56121,...,Lorient,R53,A14,D056,UU56601,56121,Lorient,Morbihan,Rennes,Bretagne
3,2023,2023-24,2024,CPGE,Classes préparatoires aux grandes écoles (CPGE),PR,Établissements privés,1,Masculin,56260,...,Vannes,R53,A14,D056,UU56501,56260,Vannes,Morbihan,Rennes,Bretagne
4,2023,2023-24,2024,CPGE,Classes préparatoires aux grandes écoles (CPGE),PR,Établissements privés,1,Masculin,57480,...,Montigny-lès-Metz,R44,A12,D057,UU57701,57480,Metz,Moselle,Nancy-Metz,Grand Est
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149573,2001,2001-02,2002,ING_autres,Autres formations d'ingénieurs,PU,Établissements publics,1,Masculin,92063,...,Rueil-Malmaison,R11,A25,D092,UU00851,92063,Paris,Hauts-de-Seine,Versailles,Île-de-France
149574,2001,2001-02,2002,ING_autres,Autres formations d'ingénieurs,PU,Établissements publics,1,Masculin,91477,...,Palaiseau,R11,A25,D091,UU00851,91477,Paris,Essonne,Versailles,Île-de-France
149575,2001,2001-02,2002,ING_autres,Autres formations d'ingénieurs,PU,Établissements publics,1,Masculin,91377,...,Massy,R11,A25,D091,UU00851,91377,Paris,Essonne,Versailles,Île-de-France
149576,2001,2001-02,2002,ING_autres,Autres formations d'ingénieurs,PU,Établissements publics,1,Masculin,91272,...,Gif-sur-Yvette,R11,A25,D091,UU00851,91272,Paris,Essonne,Versailles,Île-de-France
